<div>
<center><img src="Flux-logo.svg" width="400"/>
</div>

# Chapter 2: Flux Plumbing 💩️🚽️

> How to get to Porcelain? You start with Plumbing, of course - "the toilet vs. the pipes"

Now that we have learned about basic flux commands, and hierarchical scheduling and its benefits, let's dive deeper into the structure of the individual Flux instances that comprise a hierarchy and talk about some additional "plumbing" that helps Flux to run. In this module, we cover:
1. The structure of Flux instances
2. Flux modules
3. Examples `flux kvs` that powers a lot of higher level commands
4. Advanced job specification interaction with flux job

<br>

## The structure of Flux instances

As mentioned in [Chapter 2](./01_flux_tutorial.ipynb), a Flux instance is comprised of one or more Flux brokers. A high-level depiction of the design of a Flux broker is shown in the figure below.

<figure>
<img src="img/flux-broker-design.png">
<figcaption>
<i>Image created by Ian Lumsden for the Flux tutorials</i></figcaption>
</figure>

Each broker is a program built on top of the ∅MQ networking library. The broker contains two main components. First, the broker implements Flux-specific networking abstractions over ∅MQ, such as remote-proceedure call (RPC) and publication-subscription (pub-sub). Second, the broker contains several core services, such as PMI (for MPI support), run control support (for enabling automatic startup of other services), and, most importantly, broker module management. The remainder of a Flux broker's functionality comes from broker modules: specially designed services that the broker can deploy in independent OS threads. Some examples of broker modules provided by Flux include:
* Job scheduling (both traditional and hierarchical)
* [Fluxion](https://github.com/flux-framework/flux-sched) (Flux's advanced graph-based scheduler)
* Banks and accounting (for system-wide deployments of Flux)
* [PMIx](https://github.com/openpmix/openpmix) (for OpenMPI)
* An in-memory content store (useful for preloading data into pods on cloud)

When Flux starts, it launches one or more brokers across the resources it manages. By default, Flux will launch one broker per node, but this can be configured (e.g., with the `--test-size` flag to `flux start` shown in [Chapter 1](./01_flux_tutorial.ipynb)). After launching the brokers, Flux will designate one broker as the "leader" and the rest as "followers". The leader serves as entrypoint into the Flux instance, and it serves as the starting point for most Flux commands. The distribution of brokers and the "leader-follower" designations are shown in the following figure:

<figure>
<img src="img/flux-instance-pre-tbon.png">
<figcaption>
<i>Image created by Vanessa Sochat for Flux Framework Components documentation</i></figcaption>
</figure>

After launching the brokers and designating a leader, Flux uses the brokers' network abstractions to connect the brokers together into what we call the "tree-based overlay network" or TBON for short. This network is shown in the figure below. This overlay network connects brokers together in a pre-defined tree-based topology (e.g., *k*-ary and binomial). Whenever brokers or instances of distributed services running on top of the brokers need to communicate, they can send messages up and down this tree-structured network. This tree-structured network is used over alternative designs (e.g., all-to-all networks used by MPI) because it provides better scalability (by minimizing communication), security, and fault tolerance for a service-focused framework. More information about these benefits and Flux's overall design can be found in our [publications](https://flux-framework.org/publications/) (particularly our [2014 paper on Flux](https://ieeexplore.ieee.org/document/7103433) presented at ICPP).

<figure>
<img src="img/flux-instance-w-tbon.png">
<figcaption>
<i>Image created by Vanessa Sochat for Flux Framework Components documentation</i></figcaption>
</figure>

Flux functionality can be extended with modules, which you might think of like services. For Flux instances, additional services are typically implemented as broker modules that can be deployed across one or more brokers. Once deployed, these services can leverage the other components of the broker, including message routing over the TBON and services provided by other broker modules. As a result, broker modules allow for the creation of composable, easily deployable services for Flux instances.

<br>

## Flux Modules

To manage and query modules, Flux provides the `flux module` command. The sub-commands provided by `flux module` can be seen by running the cell below.

In [2]:
!flux module --help

Usage: flux-module COMMAND [OPTIONS]
  -h, --help             Display this message.

flux module subcommands:
   list            List loaded modules
   remove          Unload module
   load            Load module
   reload          Reload module
   stats           Display stats on module
   debug           Get/set module debug flags


Some examples of Flux modules include:
* `job-ingest` (used by Flux submission commands like `flux batch` and `flux run`)
* `job-list` (used by `flux jobs`)
* `sched-fluxion-qmanager` (used by `flux tree`)
* `sched-fluxion-resource` (also used by `flux tree`)

We can see that these services are loaded and available by running the cell below.

In [3]:
!flux module list

Module                   Idle  S Service
job-exec                 idle  R 
heartbeat                   1  R 
job-list                 idle  R 
sched-fluxion-resource   idle  R 
content-sqlite           idle  R content-backing
resource                 idle  R 
job-ingest               idle  R 
content                  idle  R 
job-info                 idle  R 
sched-fluxion-qmanager   idle  R sched
kvs-watch                idle  R 
kvs                      idle  R 
cron                     idle  R 
job-manager              idle  R 
barrier                  idle  R 
connector-local             0  R 


Users and system administrators can easily load and unload modules using the `flux module load` and `flux module remove` commands. To show this, let's unload Fluxion (Flux's graph-based scheduler) and replace it with the built-in simple scheduler.

In [4]:
!flux module remove sched-fluxion-qmanager
!flux module remove sched-fluxion-resource
!flux module load sched-simple
!flux module list

Module                   Idle  S Service
job-exec                 idle  R 
heartbeat                   0  R 
job-list                 idle  R 
content-sqlite           idle  R content-backing
resource                    0  R 
job-ingest               idle  R 
content                     0  R 
job-info                 idle  R 
kvs-watch                idle  R 
kvs                         0  R 
cron                     idle  R 
job-manager                 0  R 
sched-simple                0  R sched
barrier                  idle  R 
connector-local             0  R 


In this code block, we unload the 2 services that comprise Fluxion: `sched-fluxion-qmanager` and `sched-fluxion-resource`. Next, we load the simple scheduler (`sched-simple`), and, finally, we look at the running servicees. We now see that Fluxion is not available, and the simple scheduler is. Next, let's reload Fluxion, but, this time, let's pass some extra arguments to specialize our Flux instance. In particular, we will limit the scheduling depth to 4 and populate Fluxion's resource graph with:
* Nodes
* Sockets
* Cores

In [5]:
# Run flux dmesg to make sure sched-simple has no more work before unloading
!flux dmesg -C
!flux module remove sched-simple
!flux module load sched-fluxion-resource load-allowlist=node,socket,core
!flux module load sched-fluxion-qmanager queue-params=queue-depth=4
!flux module list

Module                   Idle  S Service
job-exec                 idle  R 
heartbeat                   1  R 
job-list                 idle  R 
sched-fluxion-qmanager      0  R sched
content-sqlite           idle  R content-backing
resource                    0  R 
job-ingest               idle  R 
content                     0  R 
job-info                 idle  R 
kvs-watch                idle  R 
sched-fluxion-resource      0  R 
kvs                         0  R 
cron                     idle  R 
job-manager                 0  R 
barrier                  idle  R 
connector-local             0  R 


### flux kvs

One of the core services built into Flux is the key-value store (KVS). It is used in many other services, including most of Flux's resource management services, the `flux archive` service below, and DYAD (which we will explore in [Supplementary Chapter 1](./supplementary/dyad/dyad_dlio.ipynb)). These services use the KVS to persistantly store information and retrieve it later (potentially after a restart of Flux).

The `flux kvs` command provides a utility to list and manipulate values of the KVS. As a example of using `flux kvs`, let's use the command to examine information saved by the `resource` service.

In [ ]:
!flux kvs ls
!flux kvs ls resource
!flux kvs get resource.R | jq

The KVS is such an essential component of Flux that we provide C and Python APIs to interact with it. To learn more about interacting with the KVS from these languages, take a look at these documentation pages:
* C's `flux_kvs_commit` [family of functions](https://flux-framework.readthedocs.io/projects/flux-core/en/latest/man3/flux_kvs_commit.html)
* C's `flux_kvs_copy` [family of functions](https://flux-framework.readthedocs.io/projects/flux-core/en/latest/man3/flux_kvs_copy.html)
* C's `flux_kvs_getroot` [family of functions](https://flux-framework.readthedocs.io/projects/flux-core/en/latest/man3/flux_kvs_getroot.html)
* C's `flux_kvs_lookup` [family of functions](https://flux-framework.readthedocs.io/projects/flux-core/en/latest/man3/flux_kvs_lookup.html)
* C's `flux_kvs_namespace_create` [family of functions](https://flux-framework.readthedocs.io/projects/flux-core/en/latest/man3/flux_kvs_namespace_create.html)
* C's `flux_kvs_txn_create` [family of functions](https://flux-framework.readthedocs.io/projects/flux-core/en/latest/man3/flux_kvs_txn_create.html)
* Python's `flux.kvs` [module](https://flux-framework.readthedocs.io/projects/flux-core/en/latest/python/autogenerated/flux.kvs.html#module-flux.kvs)

<br>

## flux jobspec generation

Underlying much interaction with jobs is the creation of job specifications. When you use the command line or Python SDK and submit from a command or script, under the hood (back to that plumbing reference) we are creating a job specification "Jobspec" that is passed further through Flux. The command `flux submit` makes it possible to provide a similar command, but instead of running it, to generate the jobspec. Let's do that now. We will generate and view a Jobspec for a simple "hello world" job. We do that by adding `--dry-run`.

In [16]:
! flux submit --dry-run echo hello potato 🥔️🍠️ > potato-job.txt
! cat potato-job.txt | jq

{
  "resources": [
    {
      "type": "slot",
      "count": 1,
      "with": [
        {
          "type": "core",
          "count": 1
        }
      ],
      "label": "task"
    }
  ],
  "tasks": [
    {
      "command": [
        "echo",
        "hello",
        "potato",
        "🥔️🍠️"
      ],
      "slot": "task",
      "count": {
        "per_slot": 1
      }
    }
  ],
  "attributes": {
    "system": {
      "duration": 0,
      "environment": {
        "SHELL": "/usr/bin/bash",
        "FLUX_MODULE_PATH": "/usr/lib/flux/modules",
        "HOSTNAME": "8660c254a8e5",
        "FLUX_START_URI": "local:///tmp/flux-iwjuLe/start",
        "NB_UID": "1000",
        "PWD": "/home/jovyan",
        "JPY_SESSION_NAME": "/home/jovyan/02_flux_framework.ipynb",
        "MANPATH": "/usr/share/man",
        "FLUX_CONNECTOR_PATH": "/usr/lib/flux/connectors",
        "_": "/usr/bin/flux",
        "HOME": "/home/jovyan",
        "LANG": "C.UTF-8",
        "FORCE_COLOR": "1",
        "PYDEVD_US

You'll notice there is a lot of content in there! At this point you could write this to file (as we did, saving to `potato-job.txt`, edit it, and provide it directly to `flux job submit` to run. Let's try that now.

In [17]:
! flux job submit ./potato-job.txt
! flux job attach $(flux job last)

ƒ3VPB8ZEqV
hello potato 🥔️🍠️


# This concludes Chapter 2.

In this module, we covered:
1. The structure of Flux instances 
2. How to load and unload modules in Flux
3. An example flux module `flux kvs`
4. Interacting with job specifications `Jobspec`s

To finish the tutorial, open [Chapter 3](./03_flux_tutorial_conclusions.ipynb).